In [5]:
from quchem.Hamiltonian_Generator_Functions import *
from quchem.Graph import *
### HAMILTONIAN start
Molecule = 'H2'
geometry = None # [('H', (0., 0., 0.)), ('H', (0., 0., 0.74))]
basis = 'cc-pvtz'#'sto-6g'


### Get Hamiltonian
Hamilt = Hamiltonian(Molecule,
                     run_scf=1, run_mp2=1, run_cisd=1, run_ccsd=1, run_fci=1,
                     basis=basis,
                     multiplicity=1,
                     geometry=geometry)  # normally None!
QubitHamiltonian = Hamilt.Get_Qubit_Hamiltonian(threshold=None, transformation='BK')
### HAMILTONIAN end

#####################################

list(QubitHamiltonian)

Psi4 calculation for H2_cc-pvtz_singlet_H2 has failed.


KeyboardInterrupt: 

In [ ]:
Hamilt.basis

In [ ]:
from quchem.Ansatz_Generator_Functions import *

new_BK = BK_Qubit_Reordering(QubitHamiltonian, Hamilt.molecule.n_electrons, Hamilt.molecule.n_qubits)
list(new_BK.Get_Re_ordered_Hamiltonian())

In [ ]:


Sec_Quant_CC_ops_ia, Sec_Quant_CC_ops_ijab, theta_parameters_ia, theta_parameters_ijab=new_BK.Get_ia_and_ijab_terms_BK_ordering(single_cc_amplitudes=None, 
                                                                                                             double_cc_amplitudes=None, 
                                                                                                             singles_hamiltonian=None,
                                                                                                             doubles_hamiltonian=None, 
                                                                                                              tol_filter_small_terms = None)
new_BK_HF_State = new_BK.BF_HF_state_REDUCED()
new_BK_HF_State

ansatz_obj.UCCSD_single_trotter_step(Sec_Quant_CC_ops_ia, Sec_Quant_CC_ops_ijab, transformation='BK')

In [ ]:
from quchem.Ansatz_Generator_Functions import *


ansatz_obj = Ansatz(Hamilt.molecule.n_electrons, Hamilt.molecule.n_qubits)



Sec_Quant_CC_ops_ia, Sec_Quant_CC_ops_ijab, theta_parameters_ia, theta_parameters_ijab=ansatz_obj.Get_ia_and_ijab_terms(single_cc_amplitudes=None, 
                                                                                                             double_cc_amplitudes=None, 
                                                                                                             singles_hamiltonian=None,
                                                                                                             doubles_hamiltonian=None, 
                                                                                                             tol_filter_small_terms = None)
ansatz_obj.UCCSD_single_trotter_step(Sec_Quant_CC_ops_ia, Sec_Quant_CC_ops_ijab, transformation='BK')

In [ ]:


# NEW_Qubit_Molecular_Hamiltonian = new_BK.Get_Reordered_Hamiltonian_2_qubits_removed(1)
# NEW_Qubit_Molecular_Hamiltonian

In [ ]:
NEW_Qubit_Molecular_Hamiltonian = new_BK.Get_Reordered_Hamiltonian_2_qubits_removed(1)
NEW_Qubit_Molecular_Hamiltonian

LOOK at:

PHYS. REV. X, **8**, 031022 (2018)

In [ ]:
NEW_ijab_terms = new_BK.Get_Reordered_CC_qubit_terms_2_qubits_removed() #ijab terms!
NEW_ijab_terms

## Ansatz


From PHYS. REV. X, **8**, 031022 (2018):

$$U = e^{-i \theta Y_{0} X_{1}}$$

- when acting on $| 01 \rangle_{HF-BK-STATE}$ 

$$U | \psi_{HF}\rangle = | \psi_{UCCSD}\rangle$$

overall we get:

$\begin{aligned} U|01\rangle &=\left(\cos (\theta) I-i \sin (\theta) Y_{0} X_{1}\right)|10\rangle \\ &=\cos (\theta)|10\rangle-\sin (\theta)|11\rangle \end{aligned}$

- note $Y|1\rangle = -i|1\rangle$

to do this in Q.C the following circuit is employed:

In [ ]:
from quchem.Simulating_Quantum_Circuit import *
from quchem.Ansatz_Generator_Functions import *
from openfermion.ops import QubitOperator

def H2_ansatz_BK(theta):
    
    HF_circ = [cirq.X.on(cirq.LineQubit(0))]
    full_exp_circ_obj = full_exponentiated_PauliWord_circuit(QubitOperator('Y0 X1', -1j), theta)
        
    UCCSD_circ = cirq.Circuit(cirq.decompose_once((full_exp_circ_obj(*cirq.LineQubit.range(full_exp_circ_obj.num_qubits())))))
    full_circuit = cirq.Circuit([*HF_circ, *UCCSD_circ.all_operations()])
    
    return full_circuit
    
circuit_form = H2_ansatz_BK(np.pi/10)    
circuit_form

In [ ]:
from quchem.Simulating_Quantum_Circuit import *

In [ ]:
n_shots= 10000


def VQE_experiment_ENERGY(THETA):
    ansatz_cirq_circuit = H2_ansatz_BK(THETA)
    VQE_exp = VQE_Experiment(NEW_Qubit_Molecular_Hamiltonian, ansatz_cirq_circuit, n_shots)
    return VQE_exp.Calc_Energy().real

In [ ]:
theta_val = np.pi

VQE_experiment_ENERGY(theta_val)

In [ ]:
import random

theta_random_input = random.uniform(0, 2*np.pi) 

In [ ]:
### optimizer
from quchem.Scipy_Optimizer import *

GG = Optimizer(VQE_experiment_ENERGY, theta_random_input, 'Nelder-Mead', store_values=True, display_iter_steps=True,
               tol=1e-5,
               display_convergence_message=True)
GG.get_env(50)
GG.plot_convergence()
plt.show()

In [ ]:
Hamilt.molecule.fci_energy

In [ ]:
from quchem.TensorFlow_Opt import *

**gradient is given by**

https://arxiv.org/pdf/1906.08728.pdf

$$\frac{\partial O(\theta)}{\partial \theta}=\left\langle\overrightarrow{0}\left|\hat{U}^{\dagger} \hat{R}_{y}^{C \dagger}(\theta+\pi / 4) \hat{V}^{\dagger} \hat{O} \hat{V} \hat{R}_{y}^{C}(\theta+\pi / 4) \hat{U}\right| \overrightarrow{0}\right\rangle -\left\langle\overrightarrow{0}\left|\hat{U}^{\dagger} \hat{R}_{y}^{C \dagger}(\theta-\pi / 4) \hat{V}^{\dagger} \hat{O} \hat{V} \hat{R}_{y}^{C}(\theta-\pi / 4) \hat{U}\right| \overrightarrow{0}\right\rangle$$

$$\frac{\partial O(\theta)}{\partial \theta} =O(\theta+\pi / 4)-O(\theta-\pi / 4)$$

In [ ]:
def VQE_experiment_ENERGY_TF(THETA):
    ansatz_cirq_circuit = H2_ansatz_BK(THETA[0])
    VQE_exp = VQE_Experiment(NEW_Qubit_Molecular_Hamiltonian, ansatz_cirq_circuit, n_shots)
    return VQE_exp.Calc_Energy().real

In [ ]:
def calc_gradient_TF(THETA):      

    Obs_PLUS = VQE_experiment_ENERGY(THETA[0] + np.pi/4)
    Obs_MINUS = VQE_experiment_ENERGY(THETA[0] - np.pi/4)
    
    gradient = Obs_PLUS - Obs_MINUS
        
    return [(gradient, THETA)]

note:
this is very SLOW as it has to run a separate experiment TWICE for each parameter before taking a step!

In [ ]:
X0 = [random.uniform(0, 2*np.pi)]

tf_opt = Tensor_Flow_Optimizer(VQE_experiment_ENERGY_TF, X0, 'Adam', calc_gradient_TF, learning_rate=0.1, beta1=0.9,
                            beta2=0.999, store_values=True, display_iter_steps=True)
tf_opt.optimize(50)
tf_opt.plot_convergence()

In [ ]:
from quchem.Adam_Optimizer import *

In [ ]:
def calc_gradient_ADAM(THETA):      

    Obs_PLUS = VQE_experiment_ENERGY(THETA + np.pi/4)
    Obs_MINUS = VQE_experiment_ENERGY(THETA - np.pi/4)
    
    gradient = Obs_PLUS - Obs_MINUS
        
    return gradient

In [ ]:
X0 = random.uniform(0, 2*np.pi)

opt_params, list_of_inputs, list_of_outputs = Adam_Opt(X0, VQE_experiment_ENERGY,
                                                       calc_gradient_ADAM, 
                                                       learning_rate=0.1, 
                                                       beta_1=0.9, 
                                                       beta_2=0.999, 
                                                       epsilon=1e-8,
                                                       max_iter=50,
                                                       disp=True,
                                                       tolerance=1e-3,
                                                       store_steps=True)

VQE_experiment_ENERGY(opt_params)

In [ ]:
import matplotlib.pyplot as plt
# % matplotlib inline
plt.figure()
plt.plot(list_of_outputs)
plt.xlabel('iterations')
plt.ylabel('objective function value')
plt.show()